### 설문조사 자료 전처리
    
    최초 작성일 : 20/04/07
    작성자 : 조예슬
    
    < 작성내용 >
        
            - 20대 여자 안면부위 선택 전처리
            
    < 수정내용 > 
            
            20/04/08
            - 20대 여자 bright 색고정 선택 전처리
            - 20대 여자 harmony 색고정 선택 전처리
            - 20대 여자 bright 색변경 선택 전처리
            - 20대 여자 harmony 색변경 선택 전처리
            
            20/04/16
            - 20대 여자가 선정한 personalcolor definition & need 선택 전처리
        
    < 출처 >
    은주언니 햣!!▼
    https://github.com/slmteruto/CAI/blob/master/jay/Statistical%20hypothesis%20testing/Survey_preprocessing.ipynb

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname='C:\Windows/Fonts/malgun.ttf').get_name()
plt.rc('font', family=font_name)
mpl.rcParams["axes.unicode_minus"] = False

import pymysql, math, time
from sklearn import preprocessing
import urllib

from IPython.display import Image

### 20대 여성 안면부위 선택

In [10]:
# 기본항목 설문결과 & 측색 안면부위 설문결과  (모집단 분리하지 않은 항목) 데이터 준비

## 밝은색 설문 결과 불러오기
element1 = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (1).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="C, D")

element1.rename(columns={element1.columns[0]:"sex",
                         element1.columns[1]:"age_class"}, inplace=True)

## 조화색 설문 결과 불러오기
element2 = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (2).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="C, D")

element2.rename(columns={element2.columns[0]:"sex",
                          element2.columns[1]:"age_class"}, inplace=True)


## 두 결과 합치기
element = pd.concat([element1, element2], join='outer')


## 인덱스 조정
element.reset_index(inplace=True)
del element['index']


## 값 변경 반복문
# sex, age = [], []
for i in range(int(element.shape[0])):
    ## sex : Male, Female 리스트 담기
    if element["sex"][i] == "남성":
        element["sex"][i] = "Male"
    else:
        element["sex"][i] = "Female"
        
    ## age_class : 10, 20, 30, 40, 50으로 리스트 담기
    if element["age_class"][i] == "10대":
        element["age_class"][i] = "10"
    elif element["age_class"][i] == "20대":
        element["age_class"][i] = "20"
    elif element["age_class"][i] == "30대":
        element["age_class"][i] = "30"
    elif element["age_class"][i] == "40대":
        element["age_class"][i] = "40"
    else:
        element["age_class"][i] = "50"


# 측색 안면부위 설문결과 (모집단 분리하지 않은 항목) 데이터 준비

## 밝은색 설문 결과 불러오기
face1 = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (1).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="I, J, K, L")
face1.rename(columns={face1.columns[0]:"region 1",
                          face1.columns[1]:"region 2",
                          face1.columns[2]:"region 3",
                          face1.columns[3]:"region 4"}, inplace=True)

## 조화색 설문 결과 불러오기
face2 = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (2).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="I, J, K, L")
face2.rename(columns={face2.columns[0]:"region 1",
                          face2.columns[1]:"region 2",
                          face2.columns[2]:"region 3",
                          face2.columns[3]:"region 4"}, inplace=True)


## 두 결과 합치기
face = pd.concat([face1, face2], join='outer')

## 인덱스 조정
face.reset_index(inplace=True)
del face['index']


## 중복 결과값 분리
region10, region11, region20, region21, region30, region31, region40, region41 = [], [], [], [], [], [], [], []
for i in range(int(face.shape[0])): # 210
    region10.append(face["region 1"][i].split()[0])
    region11.append(face["region 1"][i].split()[1])
    
    region20.append(face["region 2"][i].split()[0])
    region21.append(face["region 2"][i].split()[1])
    
    region30.append(face["region 3"][i].split()[0])
    region31.append(face["region 3"][i].split()[1])
    
    region40.append(face["region 4"][i].split()[0])
    region41.append(face["region 4"][i].split()[1])
    
    
## 데이터프레임 형식으로 변환
region10 = pd.DataFrame(region10, columns=["region_10"])
region11 = pd.DataFrame(region11, columns=["region_11"])
region20 = pd.DataFrame(region20, columns=["region_20"])
region21 = pd.DataFrame(region21, columns=["region_21"])
region30 = pd.DataFrame(region30, columns=["region_30"])
region31 = pd.DataFrame(region31, columns=["region_31"])
region40 = pd.DataFrame(region40, columns=["region_40"])
region41 = pd.DataFrame(region41, columns=["region_41"])


## 열 합치기
region = pd.concat([region10, region11, region20, region21, region30, region31, region40, region41], axis=1)

region.reset_index(inplace=True)
del region['index']

region

## 선택항목 별 부위 매칭 (①-cheek, ②-chin, ③-face, ④-eyes, ⑤-forehead)

num_list = [10, 11, 20, 21, 30, 31, 40, 41]

for j in (num_list):
    region_num = "region_" + str(j)

    for i in range(int(region.shape[0])):
        if region[region_num][i] == "①":
            region[region_num][i] = "cheek"
        elif region[region_num][i] == "②":
            region[region_num][i] = "chin"
        elif region[region_num][i] == "③":
            region[region_num][i] = "face"
        elif region[region_num][i] == "④":
            region[region_num][i] = "eyes"
        else:
            region[region_num][i] = "forehead"


SAFF = pd.concat([element, region], axis=1)

## element 부분만 가져오기

SAFF2 = SAFF.iloc[:, :2]
SAFF2

## 안면부위 선택받은 개수 세기

cheek, forehead, chin, eyes, face = [], [], [], [], []

for i in range(int(SAFF.shape[0])) :
    cheek_cnt, forehead_cnt, chin_cnt, eyes_cnt, face_cnt = [], [], [], [], []
        
    cheek_cnt = SAFF.loc[i, SAFF.loc[i] == "cheek"].count() 
    forehead_cnt = SAFF.loc[i, SAFF.loc[i] == "forehead"].count()   
    chin_cnt = SAFF.loc[i, SAFF.loc[i] == "chin"].count() 
    eyes_cnt = SAFF.loc[i, SAFF.loc[i] == "eyes"].count() 
    face_cnt = SAFF.loc[i, SAFF.loc[i] == "face"].count() 
    
    cheek.append(cheek_cnt)
    forehead.append(forehead_cnt)
    chin.append(chin_cnt)
    eyes.append(eyes_cnt)
    face.append(face_cnt)   
    
    f = pd.DataFrame([cheek, forehead, chin, eyes, face])
    face_cnt = f.T
    face_cnt.columns = ["cheek", "forehead", "chin", "eyes", "face"]
    face_cnt


result_face = pd.concat([SAFF2, face_cnt], axis=1)
# result_face
SAFF

,sex,age_class,region_10,region_11,region_20,region_21,region_30,region_31,region_40,region_41
0,Male,20,cheek,forehead,cheek,face,chin,face,cheek,forehead
1,Female,20,face,forehead,cheek,face,chin,eyes,chin,forehead
2,Female,20,chin,forehead,cheek,chin,cheek,eyes,chin,eyes
3,Female,20,chin,forehead,chin,face,cheek,eyes,face,forehead
4,Female,20,cheek,chin,chin,forehead,cheek,forehead,chin,forehead
...,...,...,...,...,...,...,...,...,...,...
205,Female,50,cheek,chin,cheek,chin,cheek,chin,cheek,chin
206,Female,50,cheek,face,cheek,forehead,face,forehead,cheek,chin
207,Female,50,cheek,chin,chin,forehead,cheek,forehead,cheek,forehead
208,Female,50,chin,forehead,cheek,chin,chin,eyes,cheek,face


In [69]:
# result_face 전처리 결과 저장
result_face.to_csv("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/result_face.csv")

### 20대 여성  색고정 bright

In [27]:
# 기본항목 설문결과 & 측색 안면부위 설문결과  (모집단 분리하지 않은 항목) 데이터 준비

## 밝은색 설문 결과 불러오기
element = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (1).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="C, D")

element.rename(columns={element.columns[0]:"sex",
                         element.columns[1]:"age_class"}, inplace=True)


## 값 변경 반복문
# sex, age = [], []
for i in range(int(element.shape[0])):
    ## sex : Male, Female 리스트 담기
    if element["sex"][i] == "남성":
        element["sex"][i] = "Male"
    else:
        element["sex"][i] = "Female"
        
    ## age_class : 10, 20, 30, 40, 50으로 리스트 담기
    if element["age_class"][i] == "10대":
        element["age_class"][i] = "10"
    elif element["age_class"][i] == "20대":
        element["age_class"][i] = "20"
    elif element["age_class"][i] == "30대":
        element["age_class"][i] = "30"
    elif element["age_class"][i] == "40대":
        element["age_class"][i] = "40"
    else:
        element["age_class"][i] = "50"


# 측색 안면부위 설문결과 (모집단 분리하지 않은 항목) 데이터 준비

## 밝은색 설문 결과 불러오기
hueFixed = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (1).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="M, N, O, P, Q, R")

hueFixed.rename(columns={hueFixed.columns[0]:"comp",
                                hueFixed.columns[1]:"triangle1",
                                hueFixed.columns[2]:"triangle2",
                                hueFixed.columns[3]:"doubleness1",
                                hueFixed.columns[4]:"doubleness2",
                                hueFixed.columns[5]:"doubleness3"}, inplace=True)

## 선택항목 별 부위 매칭 (①-채도(-5)/명도(-5), ②-채도(-5)/명도(+5), ③-original, ④-채도(+5)/명도(+5), ⑤-채도(+5)/명도(-5))
columns = ["comp", "triangle1", "triangle2", "doubleness1", "doubleness2", "doubleness3"]

for i in range(int(hueFixed.shape[0])):
    for col in columns:
        if hueFixed[col][i] == "①":
            hueFixed[col][i] = "-S-V"
        elif hueFixed[col][i] == "②":
            hueFixed[col][i] = "-S+V"
        elif hueFixed[col][i] == "③":
            hueFixed[col][i] = "original"
        elif hueFixed[col][i] == "④":
            hueFixed[col][i] = "+S+V"
        else:
            hueFixed[col][i] = "+S-V"
            
            
# 선택받은 개수 세기
sv, sV, original, SV, Sv = [], [], [], [], []

for i in range(int(hueFixed.shape[0])) :
    sv_cnt, sV_cnt, original_cnt, SV_cnt, Sv_cnt = [], [], [], [], []
    
    sv_cnt = hueFixed.loc[i, hueFixed.loc[i] == "-S-V"].count()
    sV_cnt = hueFixed.loc[i, hueFixed.loc[i] == "-S+V"].count()
    original_cnt = hueFixed.loc[i, hueFixed.loc[i] == "original"].count()
    SV_cnt = hueFixed.loc[i, hueFixed.loc[i] == "+S+V"].count()
    Sv_cnt = hueFixed.loc[i, hueFixed.loc[i] == "+S-V"].count()
    
    sv.append(sv_cnt)
    sV.append(sV_cnt)
    original.append(original_cnt)
    SV.append(SV_cnt)
    Sv.append(Sv_cnt)
    
    hF = pd.DataFrame([sv, sV, original, SV, Sv])
    hueFixed_cnt = hF.T
    hueFixed_cnt.columns = ["-S-V", "-S+V", "original", "+S+V", "+S-V"]
    hueFixed_cnt
    

# 합치기
hueFixed_bright = pd.concat([element, hueFixed_cnt], axis=1)
hueFixed_bright

,sex,age_class,-S-V,-S+V,original,+S+V,+S-V
0,Male,20,0,3,0,3,0
1,Female,20,0,2,1,2,1
2,Female,20,0,1,1,3,1
3,Female,20,0,3,0,3,0
4,Female,20,0,4,0,2,0
...,...,...,...,...,...,...,...
109,Male,20,1,0,1,4,0
110,Female,20,0,1,0,2,3
111,Female,20,1,3,0,1,1
112,Female,30,0,1,1,3,1


In [26]:
# result_hueFixed 전처리 결과 저장
hueFixed_bright.to_csv("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/hueFixed_bright.csv")

### 20대 여성  색고정 harmony 

In [32]:
# 기본항목 설문결과 & 측색 안면부위 설문결과  (모집단 분리하지 않은 항목) 데이터 준비

## 조화색 설문 결과 불러오기
element = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (2).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="C, D")

element.rename(columns={element.columns[0]:"sex",
                          element.columns[1]:"age_class"}, inplace=True)


## 값 변경 반복문
# sex, age = [], []
for i in range(int(element.shape[0])):
    ## sex : Male, Female 리스트 담기
    if element["sex"][i] == "남성":
        element["sex"][i] = "Male"
    else:
        element["sex"][i] = "Female"
        
    ## age_class : 10, 20, 30, 40, 50으로 리스트 담기
    if element["age_class"][i] == "10대":
        element["age_class"][i] = "10"
    elif element["age_class"][i] == "20대":
        element["age_class"][i] = "20"
    elif element["age_class"][i] == "30대":
        element["age_class"][i] = "30"
    elif element["age_class"][i] == "40대":
        element["age_class"][i] = "40"
    else:
        element["age_class"][i] = "50"


# 측색 안면부위 설문결과 (모집단 분리하지 않은 항목) 데이터 준비

## 조화색 설문 결과 불러오기
hueFixed = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (2).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="M, N, O, P, Q, R")

hueFixed.rename(columns={hueFixed.columns[0]:"comp",
                                hueFixed.columns[1]:"triangle1",
                                hueFixed.columns[2]:"triangle2",
                                hueFixed.columns[3]:"doubleness1",
                                hueFixed.columns[4]:"doubleness2",
                                hueFixed.columns[5]:"doubleness3"}, inplace=True)

## 선택항목 별 부위 매칭 (①-채도(-5)/명도(-5), ②-채도(-5)/명도(+5), ③-original, ④-채도(+5)/명도(+5), ⑤-채도(+5)/명도(-5))
columns = ["comp", "triangle1", "triangle2", "doubleness1", "doubleness2", "doubleness3"]

for i in range(int(hueFixed.shape[0])):
    for col in columns:
        if hueFixed[col][i] == "①":
            hueFixed[col][i] = "-S-V"
        elif hueFixed[col][i] == "②":
            hueFixed[col][i] = "-S+V"
        elif hueFixed[col][i] == "③":
            hueFixed[col][i] = "original"
        elif hueFixed[col][i] == "④":
            hueFixed[col][i] = "+S+V"
        else:
            hueFixed[col][i] = "+S-V"
            
            
# 선택받은 개수 세기
sv, sV, original, SV, Sv = [], [], [], [], []

for i in range(int(hueFixed.shape[0])) :
    sv_cnt, sV_cnt, original_cnt, SV_cnt, Sv_cnt = [], [], [], [], []
    
    sv_cnt = hueFixed.loc[i, hueFixed.loc[i] == "-S-V"].count()
    sV_cnt = hueFixed.loc[i, hueFixed.loc[i] == "-S+V"].count()
    original_cnt = hueFixed.loc[i, hueFixed.loc[i] == "original"].count()
    SV_cnt = hueFixed.loc[i, hueFixed.loc[i] == "+S+V"].count()
    Sv_cnt = hueFixed.loc[i, hueFixed.loc[i] == "+S-V"].count()
    
    sv.append(sv_cnt)
    sV.append(sV_cnt)
    original.append(original_cnt)
    SV.append(SV_cnt)
    Sv.append(Sv_cnt)
    
    hF = pd.DataFrame([sv, sV, original, SV, Sv])
    hueFixed_cnt = hF.T
    hueFixed_cnt.columns = ["-S-V", "-S+V", "original", "+S+V", "+S-V"]
    hueFixed_cnt
    

# 합치기
hueFixed_harmony = pd.concat([element, hueFixed_cnt], axis=1)
hueFixed_harmony

,sex,age_class,-S-V,-S+V,original,+S+V,+S-V
0,Female,20,1,0,2,2,1
1,Male,20,1,1,1,2,1
2,Female,20,1,2,0,1,2
3,Male,20,0,0,2,2,2
4,Female,20,0,0,0,2,4
...,...,...,...,...,...,...,...
91,Female,50,0,2,4,0,0
92,Female,50,2,1,0,1,2
93,Female,50,0,0,1,4,1
94,Female,50,0,1,2,3,0


In [33]:
# hueFixed_harmony 전처리 결과 저장
hueFixed_harmony.to_csv("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/hueFixed_harmony.csv")

### 20대 여성 색변경 bright

In [35]:
# 기본항목 설문결과 & 측색 안면부위 설문결과  (모집단 분리하지 않은 항목) 데이터 준비

## 밝은색 설문 결과 불러오기
element = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (1).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="C, D")

element.rename(columns={element.columns[0]:"sex",
                         element.columns[1]:"age_class"}, inplace=True)

## 값 변경 반복문
# sex, age = [], []
for i in range(int(element.shape[0])):
    ## sex : Male, Female 리스트 담기
    if element["sex"][i] == "남성":
        element["sex"][i] = "Male"
    else:
        element["sex"][i] = "Female"
        
    ## age_class : 10, 20, 30, 40, 50으로 리스트 담기
    if element["age_class"][i] == "10대":
        element["age_class"][i] = "10"
    elif element["age_class"][i] == "20대":
        element["age_class"][i] = "20"
    elif element["age_class"][i] == "30대":
        element["age_class"][i] = "30"
    elif element["age_class"][i] == "40대":
        element["age_class"][i] = "40"
    else:
        element["age_class"][i] = "50"

# 측색 안면부위 설문결과 (모집단 분리하지 않은 항목) 데이터 준비

## 밝은색 설문 결과 불러오기
hueChanged = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (1).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="S, T, U, V, W, X")

hueChanged.rename(columns={hueChanged.columns[0]:"comp",
                                hueChanged.columns[1]:"triangle1",
                                hueChanged.columns[2]:"triangle2",
                                hueChanged.columns[3]:"doubleness1",
                                hueChanged.columns[4]:"doubleness2",
                                hueChanged.columns[5]:"doubleness3"}, inplace=True)


columns = ["comp", "triangle1", "triangle2", "doubleness1", "doubleness2", "doubleness3"]
## 선택항목 별 부위 매칭 (①-색+72º, ②-색+144º, ③-original, ④-색+216º, ⑤-색+288º)
for i in range(int(hueChanged.shape[0])):
    for col in columns:
        if hueChanged[col][i] == "①":
            hueChanged[col][i] = "72"
        elif hueChanged[col][i] == "②":
            hueChanged[col][i] = "144"
        elif hueChanged[col][i] == "③":
            hueChanged[col][i] = "original"
        elif hueChanged[col][i] == "④":
            hueChanged[col][i] = "216"
        else:
            hueChanged[col][i] = "288"
            

# 선택받은 개수 세기
st, off, original, tos, tee = [], [], [], [], []

for i in range(int(hueChanged.shape[0])) :
    st_cnt, off_cnt, original_cnt, tos_cnt, tee_cnt = [], [], [], [], []
    
    st_cnt = hueChanged.loc[i, hueChanged.loc[i] == "72"].count()
    off_cnt = hueChanged.loc[i, hueChanged.loc[i] == "144"].count()
    original_cnt = hueChanged.loc[i, hueChanged.loc[i] == "original"].count()
    tos_cnt = hueChanged.loc[i, hueChanged.loc[i] == "216"].count()
    tee_cnt = hueChanged.loc[i, hueChanged.loc[i] == "288"].count()
    
    st.append(st_cnt)
    off.append(off_cnt)
    original.append(original_cnt)
    tos.append(tos_cnt)
    tee.append(tee_cnt)
    
    hC = pd.DataFrame([st, off, original, tos, tee])
    hueChanged_cnt = hC.T
    hueChanged_cnt.columns = ["72", "144", "original", "216", "288"]
    
    
# 합치기
hueChanged_bright = pd.concat([element, hueChanged_cnt], axis=1)
hueChanged_bright            

,sex,age_class,72,144,original,216,288
0,Male,20,0,3,1,1,1
1,Female,20,2,3,1,0,0
2,Female,20,2,3,0,1,0
3,Female,20,1,0,0,2,3
4,Female,20,2,3,0,1,0
...,...,...,...,...,...,...,...
109,Male,20,2,2,1,1,0
110,Female,20,1,1,1,3,0
111,Female,20,2,1,0,1,2
112,Female,30,1,2,1,2,0


In [36]:
# hueChanged_bright 전처리 결과 저장
hueChanged_bright.to_csv("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/hueChanged_bright.csv")

### 20대 여성 색변경 harmony

In [37]:
# 기본항목 설문결과 & 측색 안면부위 설문결과  (모집단 분리하지 않은 항목) 데이터 준비

## 조화색 설문 결과 불러오기
element = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (2).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="C, D")

element.rename(columns={element.columns[0]:"sex",
                          element.columns[1]:"age_class"}, inplace=True)

## 값 변경 반복문
# sex, age = [], []
for i in range(int(element.shape[0])):
    ## sex : Male, Female 리스트 담기
    if element["sex"][i] == "남성":
        element["sex"][i] = "Male"
    else:
        element["sex"][i] = "Female"
        
    ## age_class : 10, 20, 30, 40, 50으로 리스트 담기
    if element["age_class"][i] == "10대":
        element["age_class"][i] = "10"
    elif element["age_class"][i] == "20대":
        element["age_class"][i] = "20"
    elif element["age_class"][i] == "30대":
        element["age_class"][i] = "30"
    elif element["age_class"][i] == "40대":
        element["age_class"][i] = "40"
    else:
        element["age_class"][i] = "50"


# 측색 안면부위 설문결과 (모집단 분리하지 않은 항목) 데이터 준비

## 조화색 설문 결과 불러오기
hueChanged = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (2).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="S, T, U, V, W, X")

hueChanged.rename(columns={hueChanged.columns[0]:"comp",
                                hueChanged.columns[1]:"triangle1",
                                hueChanged.columns[2]:"triangle2",
                                hueChanged.columns[3]:"doubleness1",
                                hueChanged.columns[4]:"doubleness2",
                                hueChanged.columns[5]:"doubleness3"}, inplace=True)


columns = ["comp", "triangle1", "triangle2", "doubleness1", "doubleness2", "doubleness3"]
## 선택항목 별 부위 매칭 (①-색+72º, ②-색+144º, ③-original, ④-색+216º, ⑤-색+288º)
for i in range(int(hueChanged.shape[0])):
    for col in columns:
        if hueChanged[col][i] == "①":
            hueChanged[col][i] = "72"
        elif hueChanged[col][i] == "②":
            hueChanged[col][i] = "144"
        elif hueChanged[col][i] == "③":
            hueChanged[col][i] = "original"
        elif hueChanged[col][i] == "④":
            hueChanged[col][i] = "216"
        else:
            hueChanged[col][i] = "288"
            

# 선택받은 개수 세기
st, off, original, tos, tee = [], [], [], [], []

for i in range(int(hueChanged.shape[0])) :
    st_cnt, off_cnt, original_cnt, tos_cnt, tee_cnt = [], [], [], [], []
    
    st_cnt = hueChanged.loc[i, hueChanged.loc[i] == "72"].count()
    off_cnt = hueChanged.loc[i, hueChanged.loc[i] == "144"].count()
    original_cnt = hueChanged.loc[i, hueChanged.loc[i] == "original"].count()
    tos_cnt = hueChanged.loc[i, hueChanged.loc[i] == "216"].count()
    tee_cnt = hueChanged.loc[i, hueChanged.loc[i] == "288"].count()
    
    st.append(st_cnt)
    off.append(off_cnt)
    original.append(original_cnt)
    tos.append(tos_cnt)
    tee.append(tee_cnt)
    
    hC = pd.DataFrame([st, off, original, tos, tee])
    hueChanged_cnt = hC.T
    hueChanged_cnt.columns = ["72", "144", "original", "216", "288"]
    hueChanged_cnt
    
    
# 합치기
hueChanged_harmony = pd.concat([element, hueChanged_cnt], axis=1)
hueChanged_harmony           

,sex,age_class,72,144,original,216,288
0,Female,20,1,2,2,1,0
1,Male,20,1,1,1,1,2
2,Female,20,1,2,0,1,2
3,Male,20,0,1,0,1,4
4,Female,20,1,1,2,2,0
...,...,...,...,...,...,...,...
91,Female,50,6,0,0,0,0
92,Female,50,1,1,2,1,1
93,Female,50,0,3,0,1,2
94,Female,50,1,0,1,1,3


In [38]:
# hueChanged_harmony 전처리 결과 저장
hueChanged_harmony.to_csv("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/hueChanged_harmony.csv")

### 20대 여자가 선정한 personalcolor definition & need 선택 

In [44]:
# 데이터 불러오기

## 밝은색 설문 결과 불러오기
element1 = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (1).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="C, D, E, F")

element1.rename(columns={element1.columns[0]:"sex",
                         element1.columns[1]:"age",
                         element1.columns[2]:"definition",
                         element1.columns[3]:"need"}, inplace=True)

## 조화색 설문 결과 불러오기
element2 = pd.read_excel("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (2).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="C, D, E, F")

element2.rename(columns={element2.columns[0]:"sex",
                         element2.columns[1]:"age",
                         element2.columns[2]:"definition",
                         element2.columns[3]:"need"}, inplace=True)

## 합치기
element = pd.concat([element1, element2], join='outer')

## 인덱스 조정
element.reset_index(inplace=True)
del element['index']


# 전처리

## 필요 컬럼 생성 
element["def_harmony"], element["def_bright"], element["need_harmony"], element["need_bright"] = None, None, None, None

for i in range(int(element.shape[0])):
    ## sex
    if element["sex"][i] == "남성":
        element["sex"][i] = "Male"
    else :
        element["sex"][i] = "Female"
        
    ## age
    if element["age"][i] == "10대" :
        element["age"][i] = "10"
    elif element["age"][i] == "20대" :
        element["age"][i] = "20"
    elif element["age"][i] == "30대" :
        element["age"][i] = "30"
    elif element["age"][i] == "40대" :
        element["age"][i] = "40"
    else:
        element["age"][i] = "50"
        
    ## definition
    if "조화" in element["definition"][i] :
        element["def_harmony"][i] = 1
    elif "조화" not in element["definition"][i]:
        element["def_harmony"][i] = 0
        
    if "밝아" in element["definition"][i]:
        element["def_bright"][i] = 1
    elif "밝아" not in element["definition"][i]:
        element["def_bright"][i] = 0
        
    ## need
    if "조화" in element["need"][i] :
        element["need_harmony"][i] = 1
    elif "조화" not in element["need"][i]:
        element["need_harmony"][i] =0 
        
    if "밝아" in element["need"][i]:
        element["need_bright"][i] = 1
    elif "밝아" not in element["need"][i]:
        element["need_bright"][i] = 0
        
## 불필요 컬럼 삭제
del element["definition"]
del element["need"]

element

,sex,age,def_harmony,def_bright,need_harmony,need_bright
0,Male,20,1,0,1,0
1,Female,20,0,1,1,0
2,Female,20,0,1,0,1
3,Female,20,1,0,1,0
4,Female,20,0,1,0,1
...,...,...,...,...,...,...
205,Female,50,0,1,0,1
206,Female,50,1,0,1,0
207,Female,50,0,1,0,1
208,Female,50,1,0,1,0


In [46]:
# element 전처리 결과 저장
element.to_csv("C:/Users/cho ye seul/Documents/예슬/Acorn/FinalProject/Statistical Analysis/definition_need.csv")